# Sorting Algorithm Experiment

## Executive Summary


Set up five lists of randomly generated strings of characters (i.e. a-z or A-Z e.g. abcdefkjklkjlkjlkjkljlkjlkjlkjkljasdfgtredghjkiuyt). Each string should be 50 characters long. The first list should have 200 strings (each string should be 50 characters long), second should have 400 strings (again each string is 50 characters), third 600 (same length strings i.e. 50 characters), fourth 800 (string length continues to be 50 characters), and last/fifth 1000 strings (yes 50 characters in each string). 
You may use the code we used in previous homework assignments.  Make sure the list is unsorted and does not contain any duplicates.
Use the textbook implementation of quicksort to sort the data, being sure to capture the amount of time it takes for each list. 
Now use at least two other sorting algorithms on the same data set (e.g. merge sort, bubble sort, insertion sort, etc).  Make sure that you code the sorting algorithm and you do not use a builtin function. By this I mean you may not use np.sort() or sorted(), etc.  You must code the algorithm in order to compare the complexity of each.  Here is a good resource https://www.geeksforgeeks.org/sorting-algorithms/ (Links to an external site.)).  Capture the computation time for each list using each sorting algorithm that you have used. 

Remember to use an unsorted copy of the list each time.  Some functions alter the original list and don't return a copy.  For instance, take a look at pancake sort

arr = [23, 10, 20, 11, 12, 6, 7]
print(arr)
>> [23, 10, 20, 11, 12, 6, 7]
n = len(arr)
pancakeSort(arr, n)
print(arr)
>> [6, 7, 10, 11, 12, 20, 23]

arr now can NOT be used for another test.  Be sure to use proper copies if your choice of algorithm does this (not all do).  Reach out if you need a hand with this.

Create a table containing each algorithm and the timings for each list.  Provide a graph showing how each algorithm scales with size of list (also compare the algorithms themselves).  Discuss your findings with explanations for what you observe.  

## Data Setup

In [1]:
import numpy as np
import pandas as pd
import time
import string
import random
import matplotlib.pyplot as plt


plt.style.use('bmh')
np.random.seed = 34 #set seed
LOG = {} # Define Log

#Define Data
n = 1000
int_1 = np.random.randint(0,10000, 5*n)
int_2 = np.random.randint(0,10000, 10*n)
int_3 = np.random.randint(0,10000, 15*n)
int_4 = np.random.randint(0,10000, 20*n)
int_5 = np.random.randint(0,10000, 25*n)

float_1 = np.random.uniform(0,10000, 5*n)
float_2 = np.random.uniform(0,10000, 10*n)
float_3 = np.random.uniform(0,10000, 15*n)
float_4 = np.random.uniform(0,10000, 20*n)
float_5 = np.random.uniform(0,10000, 25*n)

string_1 = [''.join(random.choices(string.ascii_lowercase, k = 50)) for _ in range(200)]
string_2 = [''.join(random.choices(string.ascii_lowercase, k = 50)) for _ in range(400)]
string_3 = [''.join(random.choices(string.ascii_lowercase, k = 50)) for _ in range(600)]
string_4 = [''.join(random.choices(string.ascii_lowercase, k = 50)) for _ in range(800)]
string_5 = [''.join(random.choices(string.ascii_lowercase, k = 50)) for _ in range(1000)]
string_6 = [''.join(random.choices(string.ascii_lowercase, k = 50)) for _ in range(10000)]

#Setting up the different sets of arrays we created 
string_arrays = [string_1, string_2, string_3, string_4, string_5,string_6] #Raw data
int_arrays = [int_1, int_2, int_3, int_4, int_5] #Raw data
float_arrays = [float_1, float_2, float_3, float_4, float_5] #Raw data

In [2]:
string_1[:5]

['ydlzmqwmlosfbgntqyocsfmgqfgedrtkwzampqiijglwngckql',
 'qorinbakfsbgoshvymqovbniyorqutrtgvsgwxfkekmwjadcyz',
 'zjkqkcdwwuebnsaymctfncefeecgqwclucsjzorevwdajohmen',
 'kywaepsmlskeuhtqbypclkbffvondbyscgngwssprkmrpmvdgx',
 'dpcvgjpipfkmvneawpowvadcuddlxvbumkyfupfqpbtenkkdpz']

## Timing mechanism
- The timing mechanism used in this experiment wraps functions and times their overall time to return function output. 
- A dictionary "LOG" is updated with the output times of the functions

In [3]:
def timeit(log):
    """ Wraps functions needing timing provided a dictionary to dump into based on function name"""
    def log_it(func):
        def wrapped(*args, **kw):
            ts = time.time_ns() // 1000000 
            result = func(*args, **kw)
            te = time.time_ns() // 1000000 
            if func.__name__ in log.keys():
                log[func.__name__].append(te-ts)
            else:
                log[func.__name__] = [te-ts]
            return result
        return wrapped
    return log_it

## Selection Sort Algorithm

The selection sort is a comparison type sort that operates in $O(n^2)$ time complexity. It iterates through the array checking for the smallest element and then replacing that smallest element to the front of the array. The algorithm repeats with the scope of the array shortened by one element (the first smallest detected element). The algorithm will repeat until there are no elements smaller than the last smallest element indicating the sort has completed. The reason this takes $n^2$ time is because the algorithm will go through n^2 iterations for the largest element in the list, and at least one iteration for even the smallest elements. This algorithm could be implemented recursively. 

In [4]:
@timeit(LOG)
def selection_sort(arr: list) -> list:
    """
    Returns a sorted array in O(n^2) time
    ----------
        arr : list, array
            the array to sort
    """
    for i in range(0, len(arr)-1):
        small = i
        for j in range(i+1, len(arr)):
            if arr[j] < arr[small]:
                small = j
        temp_i = arr[i]
        arr[i] = arr[small]
        arr[small] = temp_i
    return arr
    


## Quick Sort Algorithm
As a comparison, I've optimized the quick sort algoirthm so that random pivots are used, greatly improving the average case. This algorithm is a good contrast to the selection sort as its average case is $O(n$ $log$ $n )$ but in the worst case it is the same as the selection sort $O(n^2)$.


In [5]:
@timeit(LOG)
def total_qsort(arr):
    def qsort(arr):
        """
        Returns a sorted array. Time-complexity = O(n log n)
        Parameters
        ----------
            arr : list, array
                the array to sort
        """
        #Base case  (already sorted)
        if len(arr) < 2:
            return arr
        #Recursive case
        else:
            pivot = arr[np.random.randint(0,len(arr))]
            less = [i for i in arr[1:] if i <= pivot]
            greater = [i for i in arr[1:] if i > pivot]
            return qsort(less) + [pivot] + qsort(greater)
    return qsort(arr)

## Insertion Sort

In [6]:
@timeit(LOG)
def insertion_sort(arr): 
    temp = arr.copy()
    # Traverse through 1 to len(arr) 
    for i in range(1, len(temp)): 
  
        key = temp[i] 
  
        # Move elements of arr[0..i-1], that are 
        # greater than key, to one position ahead 
        # of their current position 
        j = i-1
        while j >= 0 and key < temp[j] : 
                temp[j + 1] = temp[j] 
                j -= 1
        temp[j + 1] = key 
    return temp
  
  
# Driver code to test above 
arr = [12, 11, 13, 5, 6] 
s = insertion_sort(arr) 
print(arr)
print(s)


[12, 11, 13, 5, 6]
[5, 6, 11, 12, 13]


In [7]:
s

[5, 6, 11, 12, 13]

## Heap Sort

In [8]:
def heapify(arr, n, i): 
    largest = i # Initialize largest as root 
    l = 2 * i + 1     # left = 2*i + 1 
    r = 2 * i + 2     # right = 2*i + 2 
  
    # See if left child of root exists and is 
    # greater than root 
    if l < n and arr[i] < arr[l]: 
        largest = l 
  
    # See if right child of root exists and is 
    # greater than root 
    if r < n and arr[largest] < arr[r]: 
        largest = r 
  
    # Change root, if needed 
    if largest != i: 
        arr[i],arr[largest] = arr[largest],arr[i] # swap 
  
        # Heapify the root. 
        heapify(arr, n, largest) 

@timeit(LOG)
def heap_sort(arr): 
    temp = arr.copy()
    n = len(temp) 
  
    # Build a maxheap. 
    for i in range(n//2 - 1, -1, -1): 
        heapify(temp, n, i) 
  
    # One by one extract elements 
    for i in range(n-1, 0, -1): 
        temp[i], temp[0] = temp[0], temp[i] # swap 
        heapify(temp, i, 0) 
  

## Merge Sort

In [9]:
@timeit(LOG)
def total_merge_sort(arr):
    def mergeSort(arr): 
        if len(arr) >1: 
            mid = len(arr)//2 # Finding the mid of the array 
            L = arr[:mid] # Dividing the array elements  
            R = arr[mid:] # into 2 halves 

            mergeSort(L) # Sorting the first half 
            mergeSort(R) # Sorting the second half 

            i = j = k = 0

            # Copy data to temp arrays L[] and R[] 
            while i < len(L) and j < len(R): 
                if L[i] < R[j]: 
                    arr[k] = L[i] 
                    i+= 1
                else: 
                    arr[k] = R[j] 
                    j+= 1
                k+= 1

            # Checking if any element was left 
            while i < len(L): 
                arr[k] = L[i] 
                i+= 1
                k+= 1

            while j < len(R): 
                arr[k] = R[j] 
                j+= 1
                k+= 1
    return mergeSort(arr)
  

## Experiment

In [10]:
algos = [total_qsort, total_merge_sort, insertion_sort, selection_sort]

In [11]:
for arr in string_arrays:
    insertion_sort(arr)
    total_qsort(arr)
    total_merge_sort(arr)
    heap_sort(arr)
    selection_sort(arr)

In [12]:
LOG

{'insertion_sort': [0, 12, 46, 58, 103, 174, 12111],
 'total_qsort': [5, 5, 6, 5, 17, 114],
 'total_merge_sort': [2, 7, 0, 9, 14, 85],
 'heap_sort': [10, 8, 18, 21, 17, 142],
 'selection_sort': [24, 23, 49, 87, 134, 9041]}